In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : Path
    local_data_file : Path
    unzip_dir : Path

In [4]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManger:
    def __init__(
        self,
        config_filepath =CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config



In [6]:

import zipfile
import gdown
from CnnClassifier import logger
from CnnClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = f"https://drive.google.com/uc?/export=download&id={file_id}"
            gdown.download(url=prefix,output=f"artifacts/data_ingestion/data.zip")

            logger.info(f"Downloaded the data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e :
            raise e 

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e :
    raise e

[2025-10-27 09:44:13,570 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-10-27 09:44:13,575 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-10-27 09:44:13,578 : INFO : common : created directory at : artifacts]
[2025-10-27 09:44:13,581 : INFO : common : created directory at : artifacts/data_ingestion]
[2025-10-27 09:44:13,584 : INFO : 3702322914 : Downloading data from https://drive.google.com/file/d/1xcmFkbhQ_P1PisDYhSrwqeuhUizaxLlX/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1xcmFkbhQ_P1PisDYhSrwqeuhUizaxLlX
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1xcmFkbhQ_P1PisDYhSrwqeuhUizaxLlX&confirm=t&uuid=1036bc57-83df-4762-8828-a61e90131d16
To: e:\2025\Pojects\Chest-Cancer-Classification-using-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [01:02<00:00, 2.00MB/s] 

[2025-10-27 09:45:20,863 : INFO : 3702322914 : Downloaded the data from https://drive.google.com/file/d/1xcmFkbhQ_P1PisDYhSrwqeuhUizaxLlX/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
